In [1]:
!git clone https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory;pip install -e ".[torch,metrics]"
!pip install bitsandbytes>=0.39.0

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 15353, done.
remote: Counting objects: 100% (303/303), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 15353 (delta 191), reused 259 (delta 166), pack-reused 15050
Receiving objects: 100% (15353/15353), 221.71 MiB | 28.37 MiB/s, done.
Resolving deltas: 100% (11229/11229), done.
Obtaining file:///kaggle/working/LLaMA-Factory
  Installing build dependencies ... - \ | / done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... - done
  Preparing editable metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json

file_path = '/kaggle/working/LLaMA-Factory/data/dataset_info.json'

with open(file_path, 'r') as file:
    data = json.load(file)

# configure the MedQA and PubMedQA dataset information 
new_data = {
    "MedQA_train": {"file_name": "MedQA/train.json"},
    "MedQA_test": {"file_name": "MedQA/test.json"},
    "PubMedQA_pqal_train": {"file_name": "PubMedQA/pqal_train_set.json"},
    "PubMedQA_pqal_test": {"file_name": "PubMedQA/pqal_test_set.json"}
}

# update information in dataset_info.json file
data.update(new_data)
with open(file_path, 'w') as file:
    json.dump(data, file, indent=4)

In [3]:
!ls /kaggle/input/

medqa-pubmedqa


In [4]:
# create MedQA and PubMedQA directories
!mkdir -p /kaggle/working/LLaMA-Factory/data/MedQA
!mkdir -p /kaggle/working/LLaMA-Factory/data/PubMedQA

# copy MedQA dataset file
!cp /kaggle/input/medqa-pubmedqa/MedQA/train.json /kaggle/working/LLaMA-Factory/data/MedQA/
!cp /kaggle/input/medqa-pubmedqa/MedQA/test.json /kaggle/working/LLaMA-Factory/data/MedQA/

# copy PubMedQA dataset file
!cp /kaggle/input/medqa-pubmedqa/PubMedQA/pqal_train_set.json /kaggle/working/LLaMA-Factory/data/PubMedQA/
!cp /kaggle/input/medqa-pubmedqa/PubMedQA/pqal_test_set.json /kaggle/working/LLaMA-Factory/data/PubMedQA/

In [5]:
%cd LLaMA-Factory

/kaggle/working/LLaMA-Factory


In [6]:
# install packages for quantizaiton
!pip install auto_gptq>=0.5.0
!pip install optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.7/424.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.2 MB/s eta 0:00:00


max_example = 250

In [7]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --quantization_bit 8 \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 5e-05 \
    --num_train_epochs 3.0 \
    --max_samples 250 \
    --per_device_train_batch_size 6 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves\Qwen2-0.5B-Instruct\qlora\train_max_example=250 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 5 \
    --per_device_eval_batch_size 2 

2024-07-13 10:22:13.829725: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 10:22:13.829837: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 10:22:13.958490: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
07/13/2024 10:22:32 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
07/13/2024 10:22:32 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
tokenizer_config.json: 100%|███████████████| 1.29k/1.29k [00:00<00:00, 6.09MB/s]


max_example=500

In [8]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --quantization_bit 8 \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 5e-05 \
    --num_train_epochs 3.0 \
    --max_samples 500 \
    --per_device_train_batch_size 6 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves\Qwen2-0.5B-Instruct\qlora\train_max_example=500 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 5 \
    --per_device_eval_batch_size 2 

2024-07-13 10:31:44.095488: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 10:31:44.095561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 10:31:44.097028: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
07/13/2024 10:31:54 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
07/13/2024 10:31:54 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2161] 2024-07-13 10:31:54,173 >> loading file vo

max_example=1000

In [9]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --quantization_bit 8 \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 5e-05 \
    --num_train_epochs 3.0 \
    --max_samples 1000 \
    --per_device_train_batch_size 6 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves\Qwen2-0.5B-Instruct\qlora\train_max_example=1000 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 5 \
    --per_device_eval_batch_size 2 

2024-07-13 10:51:05.793924: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 10:51:05.794005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 10:51:05.795604: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
07/13/2024 10:51:15 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
07/13/2024 10:51:15 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2161] 2024-07-13 10:51:15,911 >> loading file vo